# Tutorial 5: Data Cleaning
*Also called: data pre-processing, data wrangling*

## Objectives

After this tutorial you will be able to:

*   Identify and handle missing values
*   Remove duplicates
*   Standardize data
*   Validate the cleaned data to ensure that it is accurate and complete.

<h2>Table of Contents</h2>

<ol>
    <li>
        <a href="#import">Import the dataset</a>
    </li>
    <br>
    <li>
        <details>
            <summary><a href="#clean">Identify and handle common data cleaning problems</a></summary>
            <ul>
                <li><a href="#clean-missing">Handle missing values</a></li>
                <li><a href="#clean-duplicates">Remove duplicates</a></li>
                <li><a href="#clean-standardize">Standardize data</a></li>
            </ul>
        </details>
    </li>
    <br>
    <li>
        <a href="#validate">Validate cleaned data</a>
    </li>
    <br>    
    <li>
        <a href="#outlier">Outlier Detection</a>
    </li>
    <br>    
    <li>
        <a href="#feature">Feature Extraction & Engineering</a>
    </li>
    <br>    
</ol>


<hr id="import">

<h2>1. Import the dataset</h2>

Import the `Pandas` library

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime

Read the data from `reaction_data.csv` into a `Pandas DataFrame`

In [2]:
df = pd.read_csv('reaction_data.csv')
df

,Timestamp,Reactor Type,Temperature [C],Pressure [bar],Flow Rate [l/min]
0,2023-10-21 10:00:00,CSTR,75.0,10.0,50.0
1,2023-10-21 10:00:00,CSTR,75.0,10.0,50.0
2,2023-10-21 10:00:05,CSTR,76.0,10.1,51.0
3,2023-10-21 10:00:10,PFR,77.0,10.0,NaN
4,2023-10-21 10:00:15,PFR,NaN,17.0,60.0
5,2023-10-21 10:00:20,Tubular,78.0,17.0,60.0
6,2023-10-21 10:00:25,NaN,79.0,NaN,61.0
7,2023-10-21 10:00:30,MFR,80.0,18.0,62.0


<hr id="clean">

<h2>2. Identify and handle common data cleaning problems</h2>

<h5 id="clean-missing">Handle missing values</h5>

Identify missing values

In [3]:
# get more info about the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Timestamp          8 non-null      object 
 1   Reactor Type       7 non-null      object 
 2   Temperature [C]    7 non-null      float64
 3   Pressure [bar]     7 non-null      float64
 4   Flow Rate [l/min]  7 non-null      float64
dtypes: float64(3), object(2)
memory usage: 452.0+ bytes


In [4]:
# find the number of missing values in each column
df.isna().sum()

Timestamp            0
Reactor Type         1
Temperature [C]      1
Pressure [bar]       1
Flow Rate [l/min]    1
dtype: int64

Drop rows with "NaN" from certain columns

In [5]:
# drop the rows with missing values
df.dropna(subset=['Flow Rate [l/min]'], inplace=True)
df

,Timestamp,Reactor Type,Temperature [C],Pressure [bar],Flow Rate [l/min]
0,2023-10-21 10:00:00,CSTR,75.0,10.0,50.0
1,2023-10-21 10:00:00,CSTR,75.0,10.0,50.0
2,2023-10-21 10:00:05,CSTR,76.0,10.1,51.0
4,2023-10-21 10:00:15,PFR,NaN,17.0,60.0
5,2023-10-21 10:00:20,Tubular,78.0,17.0,60.0
6,2023-10-21 10:00:25,NaN,79.0,NaN,61.0
7,2023-10-21 10:00:30,MFR,80.0,18.0,62.0


Replace "NaN" by the mean for numeric data

In [6]:
# replace the missing values with the mean of the column
avg_temp = df['Temperature [C]'].mean()
print(avg_temp)
df['Temperature [C]'].fillna(avg_temp, inplace=True)

df

77.16666666666667


,Timestamp,Reactor Type,Temperature [C],Pressure [bar],Flow Rate [l/min]
0,2023-10-21 10:00:00,CSTR,75.000000,10.0,50.0
1,2023-10-21 10:00:00,CSTR,75.000000,10.0,50.0
2,2023-10-21 10:00:05,CSTR,76.000000,10.1,51.0
4,2023-10-21 10:00:15,PFR,77.166667,17.0,60.0
5,2023-10-21 10:00:20,Tubular,78.000000,17.0,60.0
6,2023-10-21 10:00:25,NaN,79.000000,NaN,61.0
7,2023-10-21 10:00:30,MFR,80.000000,18.0,62.0


In [7]:

# impute missing values using interpolation
df['Pressure [bar]'].interpolate(method='linear', inplace=True)

df


,Timestamp,Reactor Type,Temperature [C],Pressure [bar],Flow Rate [l/min]
0,2023-10-21 10:00:00,CSTR,75.000000,10.0,50.0
1,2023-10-21 10:00:00,CSTR,75.000000,10.0,50.0
2,2023-10-21 10:00:05,CSTR,76.000000,10.1,51.0
4,2023-10-21 10:00:15,PFR,77.166667,17.0,60.0
5,2023-10-21 10:00:20,Tubular,78.000000,17.0,60.0
6,2023-10-21 10:00:25,NaN,79.000000,17.5,61.0
7,2023-10-21 10:00:30,MFR,80.000000,18.0,62.0


Replace "NaN" by the mode (for categorical data)

In [8]:
# find the unique values in the column
df['Reactor Type'].value_counts()

Reactor Type
CSTR       3
PFR        1
Tubular    1
MFR        1
Name: count, dtype: int64

In [9]:
# replace the missing values with the most frequent value
mode_type = df['Reactor Type'].mode()[0]
mode_type

'CSTR'

In [10]:
df['Reactor Type'].fillna(mode_type, inplace=True)
df

,Timestamp,Reactor Type,Temperature [C],Pressure [bar],Flow Rate [l/min]
0,2023-10-21 10:00:00,CSTR,75.000000,10.0,50.0
1,2023-10-21 10:00:00,CSTR,75.000000,10.0,50.0
2,2023-10-21 10:00:05,CSTR,76.000000,10.1,51.0
4,2023-10-21 10:00:15,PFR,77.166667,17.0,60.0
5,2023-10-21 10:00:20,Tubular,78.000000,17.0,60.0
6,2023-10-21 10:00:25,CSTR,79.000000,17.5,61.0
7,2023-10-21 10:00:30,MFR,80.000000,18.0,62.0


<h5 id="clean-duplicates">Remove duplicates</h5>

In [11]:
# find the number of duplicate rows
df.duplicated().sum()

1

In [12]:
# drop the duplicate rows
df.drop_duplicates(inplace=True)
df

,Timestamp,Reactor Type,Temperature [C],Pressure [bar],Flow Rate [l/min]
0,2023-10-21 10:00:00,CSTR,75.000000,10.0,50.0
2,2023-10-21 10:00:05,CSTR,76.000000,10.1,51.0
4,2023-10-21 10:00:15,PFR,77.166667,17.0,60.0
5,2023-10-21 10:00:20,Tubular,78.000000,17.0,60.0
6,2023-10-21 10:00:25,CSTR,79.000000,17.5,61.0
7,2023-10-21 10:00:30,MFR,80.000000,18.0,62.0


<h5 id="clean-standardize">Standardize data</h5>

In [13]:
# convert column to float
df['Temperature [C]'] = df['Temperature [C]'].astype('float64')

# convert multilple columns to float
df[['Pressure [bar]', 'Flow Rate [l/min]']] = df[['Pressure [bar]', 'Flow Rate [l/min]']].astype('float64')

# convert column to datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6 entries, 0 to 7
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Timestamp          6 non-null      datetime64[ns]
 1   Reactor Type       6 non-null      object        
 2   Temperature [C]    6 non-null      float64       
 3   Pressure [bar]     6 non-null      float64       
 4   Flow Rate [l/min]  6 non-null      float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 288.0+ bytes


In [14]:
# standardize the values in a column
df['Reactor Type'] = df['Reactor Type'].replace('Tubular', 'PFR')
df['Reactor Type'] = df['Reactor Type'].replace('MFR', 'CSTR')

# we can also use a dictionary to replace multiple values at once
df['Reactor Type'] = df['Reactor Type'].replace({
    'Tubular': 'PFR',
    'MFR': 'CSTR',
})
df

,Timestamp,Reactor Type,Temperature [C],Pressure [bar],Flow Rate [l/min]
0,2023-10-21 10:00:00,CSTR,75.000000,10.0,50.0
2,2023-10-21 10:00:05,CSTR,76.000000,10.1,51.0
4,2023-10-21 10:00:15,PFR,77.166667,17.0,60.0
5,2023-10-21 10:00:20,PFR,78.000000,17.0,60.0
6,2023-10-21 10:00:25,CSTR,79.000000,17.5,61.0
7,2023-10-21 10:00:30,CSTR,80.000000,18.0,62.0


In [15]:
# apply a function to a column       
df['Temperature [C]'] = df['Temperature [C]'].apply(lambda x: x + 273)
df.rename(columns={'Temperature [C]': 'Temperature [K]'}, inplace=True)
df

,Timestamp,Reactor Type,Temperature [K],Pressure [bar],Flow Rate [l/min]
0,2023-10-21 10:00:00,CSTR,348.000000,10.0,50.0
2,2023-10-21 10:00:05,CSTR,349.000000,10.1,51.0
4,2023-10-21 10:00:15,PFR,350.166667,17.0,60.0
5,2023-10-21 10:00:20,PFR,351.000000,17.0,60.0
6,2023-10-21 10:00:25,CSTR,352.000000,17.5,61.0
7,2023-10-21 10:00:30,CSTR,353.000000,18.0,62.0


<hr id="validate">

<h2>3. Validate cleaned data</h2>

In [16]:
# check data types
df.dtypes

Timestamp            datetime64[ns]
Reactor Type                 object
Temperature [K]             float64
Pressure [bar]              float64
Flow Rate [l/min]           float64
dtype: object

In [17]:
# check for missing values
df.isna().sum()

Timestamp            0
Reactor Type         0
Temperature [K]      0
Pressure [bar]       0
Flow Rate [l/min]    0
dtype: int64

**OR** we can use the `info()` method to check the 2 steps above at the same time

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6 entries, 0 to 7
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Timestamp          6 non-null      datetime64[ns]
 1   Reactor Type       6 non-null      object        
 2   Temperature [K]    6 non-null      float64       
 3   Pressure [bar]     6 non-null      float64       
 4   Flow Rate [l/min]  6 non-null      float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 288.0+ bytes


In [19]:
# check for duplicates
df.duplicated().sum()

0

In [20]:
# save the cleaned data for future use
df.to_csv('cleaned_data.csv', index=False)

<hr id="outlier">

<h2>4. Outlier Detection</h2>

<i style="color:yellow;">*For the following sections, load the <code>concrete_data.csv</code> dataset.</i>

In [21]:
df = pd.read_csv('concrete_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Date                                 1030 non-null   object 
 1   Cement (kg/m^3)                      1030 non-null   float64
 2   Blast Furnace Slag (kg/m^3)          1030 non-null   float64
 3   Fly Ash (kg/m^3)                     1030 non-null   float64
 4   Water (kg/m^3)                       1030 non-null   float64
 5   Superplasticizer (kg/m^3)            1030 non-null   float64
 6   Coarse Aggregate (kg/m^3)            1030 non-null   float64
 7   Fine Aggregate (kg/m^3)              1030 non-null   float64
 8   Concrete compressive strength (MPa)  1030 non-null   float64
dtypes: float64(8), object(1)
memory usage: 72.6+ KB



##### 1. **IQR Method**


In [22]:
# Define a function to detect outliers using the IQR method
def detect_outliers_iqr(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    # limit the lower limit to min value in the data
    lower_bound = max(Q1 - 1.5 * IQR, data.min())
    upper_bound = min(Q3 + 1.5 * IQR, data.max())
    return [lower_bound, upper_bound]


In [23]:

# Apply the IQR method to the 'Concrete compressive strength (MPa)' column
lower_strength, upper_strength = detect_outliers_iqr(df['Concrete compressive strength (MPa)'])
print('Concrete compressive strength:', lower_strength, upper_strength)

# count the number of outliers
outliers = df[(df['Concrete compressive strength (MPa)'] < lower_strength) | (df['Concrete compressive strength (MPa)'] > upper_strength)]
outliers

Concrete compressive strength: 2.33 79.77250000000001


,Date,Cement (kg/m^3),Blast Furnace Slag (kg/m^3),Fly Ash (kg/m^3),Water (kg/m^3),Superplasticizer (kg/m^3),Coarse Aggregate (kg/m^3),Fine Aggregate (kg/m^3),Concrete compressive strength (MPa)
0,9/29/2024,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,79.99
153,9/1/2024,323.7,282.8,0.0,183.8,10.3,942.7,659.9,80.20
181,7/28/2024,389.9,189.0,0.0,145.9,22.0,944.7,755.8,82.60
381,9/29/2024,315.0,137.0,0.0,145.0,5.9,1130.0,745.0,81.75


In [24]:

# Apply the IQR method to the 'Cement' column
lower_cement, upper_cement = detect_outliers_iqr(df['Cement (kg/m^3)'])
print('Cement:', lower_cement, upper_cement)

# count the number of outliers
# expected to be empty since there are no outliers in the 'Cement' column
outliers = df[(df['Cement (kg/m^3)'] < lower_cement) | (df['Cement (kg/m^3)'] > upper_cement)]
outliers


Cement: 102.0 540.0


,Date,Cement (kg/m^3),Blast Furnace Slag (kg/m^3),Fly Ash (kg/m^3),Water (kg/m^3),Superplasticizer (kg/m^3),Coarse Aggregate (kg/m^3),Fine Aggregate (kg/m^3),Concrete compressive strength (MPa)



##### 2. **Z-Score Method**


In [25]:
# Define a function to return the Z-Score
def detect_outliers_zscore(data):
    z = np.abs(stats.zscore(data))
    return z

# Apply the Z-Score method to the 'Superplasticizer (kg/m^3)' column
superplasticizer_zscore = detect_outliers_zscore(df['Superplasticizer (kg/m^3)'])
superplasticizer_outliers = df[superplasticizer_zscore > 3]
superplasticizer_outliers

,Date,Cement (kg/m^3),Blast Furnace Slag (kg/m^3),Fly Ash (kg/m^3),Water (kg/m^3),Superplasticizer (kg/m^3),Coarse Aggregate (kg/m^3),Fine Aggregate (kg/m^3),Concrete compressive strength (MPa)
76,10/24/2024,469.0,117.2,0.0,137.8,32.2,852.1,840.5,40.2
79,10/24/2024,531.3,0.0,0.0,141.8,28.2,852.1,893.7,41.3
99,10/20/2024,469.0,117.2,0.0,137.8,32.2,852.1,840.5,54.9
102,10/20/2024,531.3,0.0,0.0,141.8,28.2,852.1,893.7,46.9
122,9/29/2024,469.0,117.2,0.0,137.8,32.2,852.1,840.5,66.9
125,9/29/2024,531.3,0.0,0.0,141.8,28.2,852.1,893.7,56.4
145,9/1/2024,469.0,117.2,0.0,137.8,32.2,852.1,840.5,69.3
148,9/1/2024,531.3,0.0,0.0,141.8,28.2,852.1,893.7,58.8
168,7/28/2024,469.0,117.2,0.0,137.8,32.2,852.1,840.5,70.7
171,7/28/2024,531.3,0.0,0.0,141.8,28.2,852.1,893.7,59.2


<hr id="feature">

<h2>5. Feature Extraction & Engineering</h2>


##### 1. **Water-to-Cement Ratio**


In [26]:
# Create a new feature: Water-to-Cement Ratio
df['Water-to-Cement Ratio'] = df['Water (kg/m^3)'] / df['Cement (kg/m^3)']

# Display the first few rows of the new feature
print("\nWater-to-Cement Ratio:")
df.head()


Water-to-Cement Ratio:


,Date,Cement (kg/m^3),Blast Furnace Slag (kg/m^3),Fly Ash (kg/m^3),Water (kg/m^3),Superplasticizer (kg/m^3),Coarse Aggregate (kg/m^3),Fine Aggregate (kg/m^3),Concrete compressive strength (MPa),Water-to-Cement Ratio
0,9/29/2024,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,79.99,0.300000
1,9/29/2024,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,61.89,0.300000
2,1/31/2024,332.5,142.5,0.0,228.0,0.0,932.0,594.0,40.27,0.685714
3,10/28/2023,332.5,142.5,0.0,228.0,0.0,932.0,594.0,41.05,0.685714
4,11/2/2023,198.6,132.4,0.0,192.0,0.0,978.4,825.5,44.30,0.966767



##### 2. **Aggregate Ratio**


In [27]:
# Create a new feature: Aggregate Ratio (Coarse/Fine)
df['Aggregate Ratio'] = df['Coarse Aggregate (kg/m^3)'] / df['Fine Aggregate (kg/m^3)']

# Display the first few rows of the new feature
print("\nAggregate Ratio:")
print(df[['Coarse Aggregate (kg/m^3)', 'Fine Aggregate (kg/m^3)', 'Aggregate Ratio']].head())



Aggregate Ratio:
   Coarse Aggregate (kg/m^3)  Fine Aggregate (kg/m^3)  Aggregate Ratio
0                     1040.0                    676.0         1.538462
1                     1055.0                    676.0         1.560651
2                      932.0                    594.0         1.569024
3                      932.0                    594.0         1.569024
4                      978.4                    825.5         1.185221



##### 3. **Age (day)**


In [28]:
# Convert the 'Date' column to datetime format (assuming the column is named 'Date')
df['Date'] = pd.to_datetime(df['Date'])
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 11 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Date                                 1030 non-null   datetime64[ns]
 1   Cement (kg/m^3)                      1030 non-null   float64       
 2   Blast Furnace Slag (kg/m^3)          1030 non-null   float64       
 3   Fly Ash (kg/m^3)                     1030 non-null   float64       
 4   Water (kg/m^3)                       1030 non-null   float64       
 5   Superplasticizer (kg/m^3)            1030 non-null   float64       
 6   Coarse Aggregate (kg/m^3)            1030 non-null   float64       
 7   Fine Aggregate (kg/m^3)              1030 non-null   float64       
 8   Concrete compressive strength (MPa)  1030 non-null   float64       
 9   Water-to-Cement Ratio                1030 non-null   float64       
 10  Aggregate Ra

In [29]:
# Calculate the age in days as (today - date)
today = datetime.today()
df['Age (day)'] = (today - df['Date']).dt.days

# Display the first few rows with the new 'Age (day)' feature
print("\nCalculated Age in Days:")
print(df[['Date', 'Age (day)']].head())


Calculated Age in Days:
        Date  Age (day)
0 2024-09-29         28
1 2024-09-29         28
2 2024-01-31        270
3 2023-10-28        365
4 2023-11-02        360


#### **Summary of Feature Engineering**


In [30]:

# Display the summary of the new features added to the DataFrame
new_features = ['Water-to-Cement Ratio', 'Aggregate Ratio', 'Age (day)']
print("\nSummary of New Features:")
print(df[new_features].describe())



Summary of New Features:
       Water-to-Cement Ratio  Aggregate Ratio    Age (day)
count            1030.000000      1030.000000  1030.000000
mean                0.748266         1.273752    45.662136
std                 0.314005         0.185670    63.169912
min                 0.266893         0.858453     1.000000
25%                 0.533333         1.121488     7.000000
50%                 0.675349         1.266055    28.000000
75%                 0.935165         1.358146    56.000000
max                 1.882353         1.874876   365.000000



### Explanation
- **Outlier Detection**:
  - The **IQR method** identifies outliers based on the middle spread of the data.
  - The **Z-Score method** standardizes the data and flags values that deviate significantly from the mean.
- **Feature Engineering**:
  - **Water-to-Cement Ratio** and **Aggregate Ratio** represent important physical relationships in the concrete mix.
  - **Age (day)** represents the age of the concrete mix in days.
  - **Total Cementitious Materials** sums the cement, slag, and fly ash, providing an overall view of the binding materials.

<hr style="margin-top: 4rem;">
<h2>Author</h2>

<a href="https://github.com/SamerHany">Samer Hany</a>

<h2>References</h2>
<a href="https://www.w3schools.com/python/default.asp">w3schools.com</a>